In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import ipywidgets as widgets
import matplotlib.patches as mpatches
from ipywidgets import interact, interactive

In [2]:
df = pd.read_csv("/home/quantum/Downloads/agrivision_data/no_augm/train_val_evaluation.csv")

In [3]:
df.head()

,name,iou,dice,ds_part,class_name
0,2PJHLLWTV_3002-2108-3514-2620.jpg,0.0,0.0,train,cloud_shadow
1,VPDZMN9XK_1369-1716-1881-2228.jpg,0.0,0.0,train,cloud_shadow
2,MQ894WR89_6868-11377-7380-11889.jpg,0.0,0.0,train,cloud_shadow
3,PXADNP2KG_5994-766-6506-1278.jpg,0.0,0.0,train,cloud_shadow
4,7DHJZ1TBE_849-3746-1361-4258.jpg,0.0,0.0,train,cloud_shadow


In [4]:
dataset_path = "/home/quantum/PycharmProjects/agrivision_challenge/dataset/"

In [5]:
train_df = df[df["ds_part"]=="train"]

In [6]:
df["class_name"].unique()

array(['cloud_shadow', 'double_plant', 'planter_skip', 'standing_water',
       'waterway', 'weed_cluster'], dtype=object)

In [7]:
classes = df["class_name"].unique()

In [8]:
plt.rcParams.update({'font.size': 22})

In [10]:
from tqdm import tqdm_notebook as tqdm
resulting_files = []
for class_name in classes:
    class_train_df = train_df[train_df["class_name"] == class_name].reset_index(drop=True)
    for idx, row in tqdm(class_train_df[(class_train_df["iou"] > 0.0) & (class_train_df["iou"] < 0.1)].iterrows()):
        path = os.path.join(dataset_path, row["ds_part"], "labels", class_name, row["name"].replace('.jpg', '.png'))
        img = cv2.imread(path)
        if img.sum() > 0:
            resulting_files.append(row["name"])
resulting_files = np.unique(resulting_files)

In [16]:
input_dir = os.path.join(dataset_path, "train")
prediction_dir = "/home/quantum/Downloads/agrivision_data/no_augm/"
invalid_data = []
questionable_data = []
def visualize(x):
    from IPython.display import display
    button = widgets.Button(description="Add to invalid data")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        with output:
            invalid_data.append(resulting_files[x])
    button.on_click(on_button_clicked)
    
    questionable_data_button = widgets.Button(description="Add to questionable data")
    questionable_data_output = widgets.Output()
    display(questionable_data_button, questionable_data_output)
    def on_questionable_data_button_clicked(b):
        with questionable_data_output:
            questionable_data.append(resulting_files[x])
    questionable_data_button.on_click(on_questionable_data_button_clicked)
    
    rows = 3
    cols = 2
    f, axes = plt.subplots(rows, cols, figsize=(14, 21))
    total_plots = 3 * 2
    filename = resulting_files[x]
    rgb_path = os.path.join(dataset_path, "train", "images", "rgb", filename)
    rgb = cv2.cvtColor(cv2.imread(rgb_path), cv2.COLOR_BGR2RGB)
    nir_path = os.path.join(dataset_path, "train", "images", "nir", filename)
    nir = cv2.cvtColor(cv2.imread(nir_path), cv2.COLOR_BGR2RGB)
    
    boundary_path = os.path.join(input_dir, "boundaries", filename.replace('.jpg', '.png'))
    boundary = cv2.imread(boundary_path, cv2.IMREAD_GRAYSCALE).astype(bool)
    mask_path = os.path.join(input_dir, "masks", filename.replace('.jpg', '.png'))
    if os.path.exists(mask_path):
        invalid_pixels_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(bool)
    else:
        invalid_pixels_mask = np.ones(boundary.shape)
    valid_pixels_mask = np.logical_and(boundary, invalid_pixels_mask).astype(np.uint8)

    axes[2, 0].set_title("Valid pixels(black-invalid, white-valid)")
    axes[2, 0].imshow(valid_pixels_mask, 'gray')
    
    colors = [np.array([255,0,0]), 
      np.array([0,255,0]), 
      np.array([0,0,255]), 
      np.array([255,255,0]), 
      np.array([255,0,255]), 
      np.array([0,255,255])]
    
    label_stack = np.zeros(rgb.shape)
    labels_path = os.path.join(input_dir, "labels")
    for label_idx, label in enumerate(classes):
        label_mask = cv2.imread(os.path.join(labels_path, label, filename.replace('.jpg', '.png')))
        mask = cv2.cvtColor(label_mask, cv2.COLOR_BGR2GRAY)
        label_stack += label_mask / 255 * colors[label_idx]
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            for contour in contours:
                if (contour.shape[0] > 10):
                    rgb = cv2.drawContours(rgb, [contour], 0, (0,0,255), 2)
        
    for ax_idx in range(rows*cols):
        axes[ax_idx % rows, ax_idx // rows].set_xticks([])
        axes[ax_idx % rows, ax_idx // rows].set_yticks([])
        axes[ax_idx % rows, ax_idx // rows].set_aspect('equal')
    axes[0, 0].imshow(rgb)
    axes[0, 0].set_title(f"RGB, {filename}")
    dif_nir_r = (nir[:,:,0].astype(float) - rgb[:,:,0].astype(float))
    sum_nir_r = (nir[:,:,0].astype(float) + rgb[:,:,0].astype(float))
    dif_nir_g = (nir[:,:,0].astype(float) - rgb[:,:,1].astype(float))
    sum_nir_g = (nir[:,:,0].astype(float) + rgb[:,:,1].astype(float))
    ndvi = ((dif_nir_r / sum_nir_r + 1) * 127).astype(np.uint8)
    ndwi = ((dif_nir_g / sum_nir_g + 1) * 127).astype(np.uint8)
    axes[0, 1].imshow(nir)
    axes[0, 1].set_title("nir")
    axes[1, 1].imshow(ndvi)
    axes[1, 1].set_title("ndvi")
    axes[1, 0].imshow(ndwi)
    axes[1, 0].set_title("ndwi")
    patches = [ mpatches.Patch(color=np.append(colors[i] / 255, 1), label=classes[i]) for i in range(len(classes)) ]
    axes[2, 1].legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )
    axes[2, 1].set_title("Ground truth")
    axes[2, 1].imshow(label_stack, 'gray')
    
plt.subplots_adjust(wspace=0, hspace=0)

interact(visualize, x=widgets.IntSlider(min=0, 
                                max=len(resulting_files) - 1, 
                                step=1, 
                                value=0, 
                                continuous_update=False));


<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=1002), Output()), _dom_…

In [26]:
len(questionable_data), len(invalid_data)

(283, 230)

In [27]:
questionable_data

['11UBQLX1B_902-7506-1414-8018.jpg',
 '13MQGW9CZ_820-1231-1332-1743.jpg',
 '17XVPGGZ1_4167-8556-4679-9068.jpg',
 '1L7F6F74T_3306-1744-3818-2256.jpg',
 '1W6YEHJJU_11424-5129-11936-5641.jpg',
 '1W6YEHJJU_1234-8704-1746-9216.jpg',
 '24M49T6CC_8341-2781-8853-3293.jpg',
 '24M49T6CC_9365-2269-9877-2781.jpg',
 '26DRBUF64_563-1460-1075-1972.jpg',
 '26DRBUF64_621-1173-1133-1685.jpg',
 '2CTF1YW2N_1677-834-2189-1346.jpg',
 '2CTF1YW2N_1933-920-2445-1432.jpg',
 '2CTF1YW2N_2215-897-2727-1409.jpg',
 '2CTF1YW2N_2745-875-3257-1387.jpg',
 '2CTF1YW2N_2846-1359-3358-1871.jpg',
 '2CTF1YW2N_2954-999-3466-1511.jpg',
 '2CTF1YW2N_7544-3169-8056-3681.jpg',
 '2J7111V4A_3209-1272-3721-1784.jpg',
 '2J7111V4A_5761-3743-6273-4255.jpg',
 '2U827769J_5676-5741-6188-6253.jpg',
 '3A8MTTLYD_935-4128-1447-4640.jpg',
 '3N84YAMUW_4694-865-5206-1377.jpg',
 '3N9X643LR_799-5793-1311-6305.jpg',
 '3RMCTCKVD_3856-3195-4368-3707.jpg',
 '4AI2XJDRL_1348-6920-1860-7432.jpg',
 '6DLUEEKR4_760-5219-1272-5731.jpg',
 '6GDCTNWXY_4820-4278-5

In [47]:
q_data = np.unique(questionable_data)
i_data = np.unique(invalid_data)

In [48]:
len(q_data), len(i_data)

(276, 226)

In [49]:
df_with_corrected_markup = pd.DataFrame()
df_with_corrected_markup["name"] = resulting_files
df_with_corrected_markup["questionable"] = False
df_with_corrected_markup["incorrect"] = False
df_with_corrected_markup["invalid"] = False

In [50]:
df_with_corrected_markup.loc[df_with_corrected_markup["name"].isin(q_data), "questionable"] = True
df_with_corrected_markup.loc[df_with_corrected_markup["name"].isin(i_data), "incorrect"] = True
df_with_corrected_markup["invalid"] = df_with_corrected_markup["questionable"] | df_with_corrected_markup["incorrect"]

In [51]:
df_with_corrected_markup

,name,questionable,incorrect,invalid
0,11UBQLX1B_1052-819-1564-1331.jpg,False,False,False
1,11UBQLX1B_775-8085-1287-8597.jpg,False,False,False
2,11UBQLX1B_902-7506-1414-8018.jpg,True,False,True
3,12GWAPNV1_1094-7961-1606-8473.jpg,False,False,False
4,13MQGW9CZ_606-2023-1118-2535.jpg,False,False,False
5,13MQGW9CZ_8079-607-8591-1119.jpg,False,False,False
6,13MQGW9CZ_820-1231-1332-1743.jpg,True,False,True
7,17XVPGGZ1_4167-8556-4679-9068.jpg,True,False,True
8,1L7F6F74T_3173-3201-3685-3713.jpg,False,False,False
9,1L7F6F74T_3306-1744-3818-2256.jpg,True,False,True


In [52]:
df_with_corrected_markup.to_csv("validated_low_ious.csv", index=None)

In [57]:
df = pd.read_csv("/home/quantum/PycharmProjects/agrivision_challenge/dataset/train_val_initial_2020-03-25.csv")
df = df[df["ds_part"] == "train"]
len(df)

12901

In [59]:
len(df) - df_with_corrected_markup["invalid"].sum()

12447